In [1]:
import pandas as pd 
import numpy as np
import re
import os
import shutil
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("expand.frame_repr",False)
print('import succed!')

import succed!


In [2]:
input_path = r'/Users/edz/Documents/yandi_data/AI_data'
input_path_file = r'/patient3.csv'

In [3]:
df_ym= pd.read_csv(input_path + input_path_file,sep=',',index_col=None,header=None)
print(df_ym.shape)   # 现有库存数据中“影像描述”不为空的数据容量

(250625, 16)


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 修改列名
df_ym.columns =  ['来自检查id','送阅时间','病例医院名称','阅片医院名称','病历医生id','病例医生名称','阅片医生id','阅片医生名称',
                '病史','全身病','病例诊断','影像描述','阅片诊断','质控影像描述','质控阅片诊断','主诉、病例检查、处理、处方json串']
df_ym.tail(2) #校验列名修改成功

,来自检查id,送阅时间,病例医院名称,阅片医院名称,病历医生id,病例医生名称,阅片医生id,阅片医生名称,病史,全身病,病例诊断,影像描述,阅片诊断,质控影像描述,质控阅片诊断,主诉、病例检查、处理、处方json串
250623,11018815,2019-03-04 14:54:11,河北省保定市东风社区卫生服务中心,至真阅片中心,4752,白月红,82,白海霞,NaN,NaN,NaN,双眼视盘边清色正，盘沿形态未见明确异常；视网膜血管走行正常；黄斑区未见明显异常。,双眼眼底未见明显异常,NaN,NaN,"{""complain"":"""",""impression"":"""",""deal"":"""",""pres..."
250624,11018943,2019-03-07 09:09:27,昆明市西山区永昌社区卫生服务中心,至真阅片中心,65,李玲玲,12,崔馨,NaN,NaN,NaN,双眼视盘边清色正，盘沿形态未见明确异常；左眼及右眼可见部分视网膜血管走行正常，未见异常新生血...,右眼眼底未见明显异常；左眼眼底窥不清,NaN,NaN,"{""complain"":"""",""impression"":"""",""deal"":"""",""pres..."


In [5]:
df_ym.duplicated(subset='来自检查id').sum()  #判断指定Series的行数据是否存在重复项
# 或者 df_qgy['来自检查id'].duplicated().sum() 

0

In [6]:
df_ym['影像描述'] = df_ym['影像描述'].fillna('nan') # 空缺项填充nan
df_ym['影像描述'].isnull().sum()

0

In [7]:
# df_ym[df_ym['影像描述'].str.contains('双眼视盘边清色正',re.I)]['影像描述'].value_counts()  

In [8]:
# 自定义”全角转半角“函数
def strQ2B(ustring):
    """把字符串全角转半角"""
    ss = []
    for s in ustring:   # ustring=['后发障','AION','无法诊断'],s=list中依次取得的各个字符串
        rstring = ""
#         print("每一行值s:{}".format(s)) 
        for uchar in s:  # s='后发障',uchar='后发障'中依次取得的各个字符
            inside_code = ord(uchar)
#                 print(inside_code)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
#             print(rstring)
        ss.append(rstring)
#         print(ss)
    return ss
#     return ''.join(ss)

In [9]:
#调用函数，实现全角转半角
df_ym['影像描述'] = strQ2B(df_ym['影像描述'])  # ''.join用于合并字符，df_ym.at['影像描述'] = ''.join(strQ2B(str_i))
df_ym.loc[:5,'影像描述']

0    视盘边清色淡,右眼C/D=0.7,颞侧盘沿消失,双眼黄斑中心光未见,视网膜动脉细,反光强,交...
1    双眼视盘边清色正,盘沿形态未见明确异常;视网膜血管走行正常,未见异常新生血管;右眼视网膜散在...
2    双眼视盘边清色正,盘沿形态未见明确异常;视网膜动脉管径普遍变细,A/V=1:3,视网膜动脉管...
3                                  双眼后极部散在点片状出血灶,黄白色病灶
4                          可见双眼视盘边清色淡,黄斑中心光未见,视网膜呈豹纹状。
5               双眼视盘边界清,色正常,视网膜血管走行正常,未见出血及渗出,中心凹反射不清。
Name: 影像描述, dtype: object

In [10]:
# 插入'影描_gai'列
df_ym.insert(df_ym.columns.get_loc('影像描述')+1,'影描_gai',df_ym['影像描述'])

## 去除空格、括号、波浪线等
\s表示空格，b = re.compile('\s')表示匹配空格；df_tw.at[i,'影描_gai'] = re.compile('\s').sub('',str_i)
匹配到空格之后替换为“”（即：去除空格）

In [11]:
# 去除字符串中的“\n|\t|\r|~|——”
def kong_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
    #     match_str = '(左|右\d期)' 
        if str_i == 'nan':
            pass
        elif len(re.compile('\n|\t|\r|~|——|(|)|,|、|。|\?').findall(str_i)) > 0: # 去除字符串中的“\n|\t|\r|~|——”
#             print(str_i)
            str_i_1 = str_i.replace('\n','').replace('\r','').replace('\t','').replace(',','').replace('。','').replace('、','')    # 去除\n|\t|\r和,以及。   
            str_i_2 = str_i_1.replace('~','-').replace('——','-')  # 改~和————为短中线
            str_i_3 = re.compile('\?').sub('',str_i_2)
            str_i_4 = re.compile(';').sub('',str_i_3)
            df.at[i,'影描_gai'] = str_i_4.replace('(','').replace(')','')  # 去除括号
#             print('__{}'.format(df.at[i,'影描_gai']))
        else:
            pass

    check_df = df[df['影描_gai'].str.contains('\n|\t|\r|~|——|\(|\)|,|、|。|\?|;')]
    if check_df.shape[0] > 0:
        print("剩余{}条数据存在无效标点".format(check_df.shape[0]))
        kong_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [12]:
kong_fuc(df_ym)    # 调用去除无效标点的函数

转化结束！！！


In [13]:
# 去除空格
def kong1_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('\s').findall(str_i)) > 0:
    #         print(str_i)
            df.at[i,'影描_gai'] = re.compile('\s').sub('',str_i)  # 去除字符串首尾的空格
    #         print('__{}'.format(df.at[i,'影描_gai']))
        else:
            pass

    check_df = df[df['影描_gai'].str.contains('\s')]  #校验
    if check_df.shape[0] > 0:
        print("剩余{}条数据存在空格".format(check_df.shape[0]))
        kong1_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [14]:
kong1_fuc(df_ym)

转化结束！！！


In [15]:
########### 校验是否还存在空格
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if str_i == 'nan':
        pass
    elif len(re.compile('\n|\t|\r|~|——|\(|\)|,|、|。|\?|;').findall(str_i)) > 0:
        print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('\?').sub('',str_i)  # 去除字符串首尾的空格
        print('__{}'.format(df_ym.at[i,'影描_gai']))
    else:
        pass

In [16]:
# 增加“分隔符”的自定义函数
def add_function(x):
    import re
    list_x=[i.start() for i in re.finditer('双|左|右', x)]
#     print(list_x)

    for i in range(len(list_x)):
        x = list(x)
    #     print(list_x[i])
        x.insert(list_x[i]+i,'!')
        x ="".join(x)
#     print(x)
    return x

In [17]:
#调用函数，增加分隔符
for m in range(df_ym.shape[0]):
    df_ym.at[m,'影描_gai'] = add_function(df_ym.at[m,'影描_gai'])

print(df_ym.loc[:5,'影描_gai'])  #校验

0    视盘边清色淡!右眼C/D=0.7颞侧盘沿消失!双眼黄斑中心光未见视网膜动脉细反光强交叉征+余...
1    !双眼视盘边清色正盘沿形态未见明确异常视网膜血管走行正常未见异常新生血管!右眼视网膜散在玻璃...
2    !双眼视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光...
3                                  !双眼后极部散在点片状出血灶黄白色病灶
4                            可见!双眼视盘边清色淡黄斑中心光未见视网膜呈豹纹状
5                   !双眼视盘边界清色正常视网膜血管走行正常未见出血及渗出中心凹反射不清
Name: 影描_gai, dtype: object


In [18]:
## 缺少左右眼的——添加“双”标签
# 处理字符串没有“左/右/双”的情形
def lr_shuang_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split('!',str_i)
            str_i_list = list(filter(None,str_tempt))
            str_new_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('(右|左|双)').findall(str_i_list[j])) == 0:
#                     print(str_i_list)
                    str_new_list.append('双' + str_i_list[j])

                else:
                    str_new_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(str_new_list[m] for m in range(len(str_new_list)))
#             print("___{}".format(df.at[i,'影描_gai']))

In [19]:
lr_shuang_fuc(df_ym)  ## 调用添加“双”标签的函数

In [20]:
############## 验证“双”补全
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if str_i == 'nan':
        pass
    else:
        str_tempt = re.split('!',str_i)
        str_i_list = list(filter(None,str_tempt))
        str_new_list = []
        for j in range(len(str_i_list)):
            if len(re.compile('(双|左|右)').findall(str_i_list[j])) == 0:
                print(str_i_list[j])

In [21]:
print("影描_gai——的规格：{}".format(df_ym['影描_gai'].shape[0]))
print("影像描述——的规格：{}".format(df_ym['影像描述'].shape[0]))

影描_gai——的规格：250625
影像描述——的规格：250625


# 通用字符串的名称归一化

In [22]:
# 定义归一化函数——单个标签替换
def single_replace_fuc(patten_x, repl_x,string_x):
    strinfo_0 = re.compile(patten_x, re.I)
    if len(strinfo_0.findall(string_x)) > 0:
#         print(string_x)
        string_x = re.sub(re.compile(patten_x, flags=re.I), repl_x, string_x)
#         print("__{}".format(string_x))
    else:
        pass
    
    return string_x

In [23]:
####定义”双眼\|右眼\|左眼“转为”双|左|右“的转化函数
#双、左、右眼——名称统一
# 为解决调用一次函数，会出现的“修改不完全的问题”，可以多次调用函数处理遗留的未匹配数据。
def lrs_normal_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_i_1 = single_replace_fuc('双眼', '双', str_i)  #str_i_1/2/3均为中间变量,调用函数 single_replace_fuc
            str_i_2 = single_replace_fuc('左眼', '左', str_i_1)
            str_i_3 = single_replace_fuc('右眼', '右', str_i_2)
            df.at[i,'影描_gai'] =  str_i_3

    check_df = df[df['影描_gai'].str.contains('双眼|右眼|左眼')]
    if check_df.shape[0] > 0:
        print("剩余{}条数据包含双眼|右眼|左眼".format(check_df.shape[0]))
        lrs_normal_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [24]:
lrs_normal_fuc(df_ym)    # 调用“”双眼\|右眼\|左眼“转为”双|左|右“的转化函数

剩余49416条数据包含双眼|右眼|左眼
剩余8条数据包含双眼|右眼|左眼
转化结束！！！


### C/D的名称归一化

In [25]:
# 视网膜动静脉比约1:2 动静脉管径比1:3 A/V=1:2 管径比约2:3 动静脉管径比约1:2 视网膜动静脉比例约1:2 A:V=1:2 A/V可达1:2

In [26]:
df_ym[df_ym['影描_gai'].str.contains('\d:\d',re.I)]['影描_gai']

2         双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光增强...
20        双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强...
33        双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强...
36        双底像模糊隐见视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉...
38        双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强...
50        双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区散在玻...
106                 双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区形态可
1866                双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=2:3!双黄斑区形态可
2735      双C/D=0.4-0.5!右颞上方盘沿偏窄!左盘沿形态可!双A:V=1:2黄斑区未见明显出血...
3590      双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出!双黄斑周围可见玻璃膜疣!右黄斑...
3591                   双视盘边清色红C/D=0.2A:V=1:2未见明确出血及渗出黄斑基本正常
3783      双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V可达1:2!右黄斑区可见处...
3789      双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强...
3792      双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强...
3902      双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强...
3938      双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强...
3943      双视盘边清盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=

In [27]:
####定义C/D拼写错误的转化函数
#C/D拼写错误——名称统一
def normal_cd_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            a  = 'A/V='
            strinfo_0 = re.compile('A/V=', re.I)
            strinfo_1 = re.compile('视网膜动静脉比例约')
            strinfo_2 = re.compile('视网膜动静脉比约')
            strinfo_3 = re.compile('视网膜动静脉比例')
            strinfo_4 = re.compile('视网膜动静脉比')
            strinfo_5 = re.compile('动静脉管径比约')
            strinfo_6 = re.compile('动静脉管径比')   # re.I 表示忽略大小写
            strinfo_7 = re.compile('管径比约')
            strinfo_8 = re.compile('A/V可达', re.I)
            strinfo_9 = re.compile('a:v=|a/v=')
            strinfo_10 = re.compile('\d:\d')
            
            if len(strinfo_1.findall(str_i))>0 and len(strinfo_10.findall(str_i))>0:  # 均是汉字的匹配不能加re.I,否则识别不出来
#                 print(str_i)
                df.at[i,'影描_gai'] = strinfo_1.sub(a,str_i)
#                 print("~~~~~"+df.at[i,'影描_gai'])
            elif len(strinfo_2.findall(str_i,re.I))>0 and len(strinfo_10.findall(str_i))>0:  # 
#                 print(str_i)
                df.at[i,'影描_gai'] = strinfo_2.sub(a,str_i)
#                 print("======"+df.at[i,'影描_gai']) 
            elif len(strinfo_3.findall(str_i,re.I))>0 and len(strinfo_10.findall(str_i))>0:  # 
#                 print(str_i)
                df.at[i,'影描_gai'] = strinfo_3.sub(a,str_i)
#                 print("======"+df.at[i,'影描_gai']) 
            elif len(strinfo_4.findall(str_i,re.I))>0 and len(strinfo_10.findall(str_i))>0:  # 
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_4.sub(a,str_i)
                print("======"+df.at[i,'影描_gai']) 
            elif len(strinfo_5.findall(str_i,re.I))>0 and len(strinfo_10.findall(str_i))>0:  # 
#                 print(str_i)
                df.at[i,'影描_gai'] = strinfo_5.sub(a,str_i)
#                 print("======"+df.at[i,'影描_gai']) 
            elif len(strinfo_6.findall(str_i,re.I))>0 and len(strinfo_10.findall(str_i))>0:  # 
#                 print(str_i)
                df.at[i,'影描_gai'] = strinfo_6.sub(a,str_i)
#                 print("======"+df.at[i,'影描_gai']) 
            elif len(strinfo_7.findall(str_i,re.I))>0 and len(strinfo_10.findall(str_i))>0:  # 
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_7.sub(a,str_i)
                print("======"+df.at[i,'影描_gai'])
            elif len(strinfo_8.findall(str_i,re.I))>0 and len(strinfo_10.findall(str_i))>0:  # 
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_8.sub(a,str_i)
                print("======"+df.at[i,'影描_gai'])
            elif len(strinfo_9.findall(str_i,re.I))>0 and len(strinfo_10.findall(str_i))>0:  # 
                print(str_i)
                df.at[i,'影描_gai'] = strinfo_9.sub(a,str_i)
                print("======"+df.at[i,'影描_gai'])
            else:
                pass

    check_df_jb_0 = df[df['影描_gai'].str.contains('A/V')] 
    check_df_jb = check_df_jb_0[check_df_jb_0['影描_gai'].str.contains('视网膜动静脉比例约|视网膜动静脉比约|管径比约|管径比|a:v=|a/v=')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
    if check_df_jb.shape[0] > 0:
        print("剩余{}条数据未转化为".format(check_df_jb.shape[0]))
        print(check_df_jb[['影像描述','影描_gai']])
        normal_cd_fuc(df)
    else:
        print('转化结束！！！')         

In [28]:
normal_cd_fuc(df_ym)

双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V可达1:2!右黄斑区可见处未见明显异常!左玻璃体混浊遮挡黄斑黄斑区可见处未见明显异常
======双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V=1:2!右黄斑区可见处未见明显异常!左玻璃体混浊遮挡黄斑黄斑区可见处未见明显异常
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V可达1:3视网膜动脉管壁反光增强黄斑区结构未见明显异常
======双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V=1:3视网膜动脉管壁反光增强黄斑区结构未见明显异常
双视盘边清色正盘沿形态未见明确异常!双视网膜动脉管径普遍变细!左A/V可达1:3!双黄斑区结构未见明显异常
======双视盘边清色正盘沿形态未见明确异常!双视网膜动脉管径普遍变细!左A/V=1:3!双黄斑区结构未见明显异常
双视盘边清色可C/D=0.3a:v=2:3黄斑区形态可细节窥不清
======双视盘边清色可C/D=0.3A/V=2:3黄斑区形态可细节窥不清
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V可达1:2黄斑区可见散在玻璃膜疣余未见明显异常
======双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V=1:2黄斑区可见散在玻璃膜疣余未见明显异常
双视盘界清色可动静脉走行正常管径比约2:3黄斑区未见明显异常
======双视盘界清色可动静脉走行正常A/V=2:3黄斑区未见明显异常
双视盘界清色可动静脉走行正常管径比约2:3黄斑区未见明显异常
======双视盘界清色可动静脉走行正常A/V=2:3黄斑区未见明显异常
双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V可达1:2!右黄斑区结构未见明显异常!左黄斑区模糊可见处未见明显异常
======双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V=1:2!右黄斑区结构未见明显异常!左黄斑区模糊可见处未见明显异常
双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V可达1:3!右黄斑区大致正常细节窥不清!左黄斑区及周边可见散在玻璃膜疣余未见明显异常
======双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V

In [29]:
check_df_jb_0 = df_ym[df_ym['影描_gai'].str.contains('A/V')] 
check_df_jb = check_df_jb_0[~check_df_jb_0['影描_gai'].str.contains('A/V=')] 

In [30]:
check_df_jb['影描_gai']

39329     双视网膜界清色正常盘沿形态可视网膜动脉管径变细A/V比约为1:2未见明显新生血管黄斑区形态大...
74731                    右视乳头颞侧色略淡!左视乳头色浅眼底动脉细A/V压迹明显未见出血渗出
86514                     双视盘色好界清!右颞上网膜片状出血波及黄斑眼底动脉细A/V压迹明显
121637                      双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V
125560                      双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V
157790    双视乳头色淡曝光强!双动脉管径细A/V1:2动脉反光强!右颞下血管弓旁可见出血!双视网膜可见...
201672                  左底像模糊!双视盘边清色正未见RNFLD眼底动脉变细反光增强A/V压迫
219532    双底像模糊!左隐见视盘边清色淡视网膜动脉管径细A/V为1:3未见异常新生血管余窥不清!右底无法判读
229565                          双视盘边清色正动脉发光增强A/V约1:3动静脉交叉压迫
238968                      双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V
Name: 影描_gai, dtype: object

In [31]:
df_ym[df_ym['影描_gai'].str.contains('视网膜动静脉比.*\d+')]['影描_gai']

Series([], Name: 影描_gai, dtype: object)

## A/V的数值标准化

In [32]:
df_ym[df_ym['影描_gai'].str.contains('A/V',re.I)]['影描_gai'].value_counts()  

双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常                                                             104
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常                                                                         48
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区结构未见明显异常                                                                                    29
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2动静脉出现交叉压迹现象黄斑区结构未见明显异常                                                                          8
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构不清                                                                   5
双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区结构未见明显异常                                                                                3
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V                                                                                                    3
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常 

In [33]:
# 纠正“A/V比约为”的情形以及”A/V为“、”A:V=“
def nornal_num_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if len(re.compile('A/V比约为').findall(str_i))>0:
            print(str_i)
            df.at[i,'影描_gai'] = re.compile('A/V比约为').sub('A/V=',str_i)
            print("__" + df.at[i,'影描_gai'])
        elif len(re.compile('A/V为').findall(str_i))>0:  # C/D=0.0.5
            print(str_i)
            df.at[i,'影描_gai'] = re.compile('A/V为').sub('A/V=',str_i)
            print("__" + df.at[i,'影描_gai'])
        elif len(re.compile('A/V约').findall(str_i))>0:
            print(str_i)
            df.at[i,'影描_gai'] = re.compile('A/V约').sub('A/V=',str_i)
            print("===" + df.at[i,'影描_gai'])
        elif len(re.compile('A/V').findall(str_i))>0 and len(re.compile('A/V=').findall(str_i))==0:
            if len(re.compile('\d:\d').findall(str_i))>0:
                print(str_i)
                df.at[i,'影描_gai'] = re.compile('A/V').sub('A/V=',str_i)
                print("===" + df.at[i,'影描_gai'])
            else:
                pass
        elif len(re.compile('A:V约').findall(str_i))>0 and len(re.compile('\d:\d').findall(str_i))>0:  # A:V约1:3
            print(str_i)
            df.at[i,'影描_gai'] = re.compile('A:V约').sub('A/V=',str_i)
            print("===" + df.at[i,'影描_gai'])
        elif len(re.compile('A:V=').findall(str_i))>0:
            print(str_i)
            df.at[i,'影描_gai'] = re.compile('A:V=').sub('A/V=',str_i)
            print("===" + df.at[i,'影描_gai'])
        elif len(re.compile('A:V').findall(str_i))>0 and len(re.compile('A:V=').findall(str_i))==0:  # A:V1:2
            if len(re.compile('\d:\d').findall(str_i))>0:
                print(str_i)
                df.at[i,'影描_gai'] = re.compile('A:V').sub('A/V=',str_i)
                print("===" + df.at[i,'影描_gai'])
            else:
                pass
        elif len(re.compile('动静脉比').findall(str_i))>0 and len(re.compile('A/V=').findall(str_i))==0:  # 动静脉比1:3
            if len(re.compile('\d:\d').findall(str_i))>0:
                print(str_i)
                df.at[i,'影描_gai'] = re.compile('动静脉比').sub('A/V=',str_i)
                print("===" + df.at[i,'影描_gai'])
            else:
                pass
        else:
            pass

In [34]:
nornal_num_fuc(df_ym)

双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区形态可
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:2!双黄斑区形态可
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=2:3!双黄斑区形态可
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=2:3!双黄斑区形态可
双C/D=0.4-0.5!右颞上方盘沿偏窄!左盘沿形态可!双A:V=1:2黄斑区未见明显出血渗出周边视网膜未见明显出血渗出
===双C/D=0.4-0.5!右颞上方盘沿偏窄!左盘沿形态可!双A/V=1:2黄斑区未见明显出血渗出周边视网膜未见明显出血渗出
双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出!双黄斑周围可见玻璃膜疣!右黄斑颞上方可见视网膜前膜
===双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出!双黄斑周围可见玻璃膜疣!右黄斑颞上方可见视网膜前膜
双视盘边清色红C/D=0.2A:V=1:2未见明确出血及渗出黄斑基本正常
===双视盘边清色红C/D=0.2A/V=1:2未见明确出血及渗出黄斑基本正常
双视盘界清色可C/D=0.4动脉细A:V=1:2后极部散在点片出血!右明显偶见玻璃膜疣黄斑大致正常
===双视盘界清色可C/D=0.4动脉细A/V=1:2后极部散在点片出血!右明显偶见玻璃膜疣黄斑大致正常
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区及周边视网膜未见明显出血渗出
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:2!双黄斑区及周边视网膜未见明显出血渗出
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区形态可
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:2!双黄斑区形态可
左玻璃体闪辉样结晶!双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区形态可
===左玻璃体闪辉样结晶!双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:2!双黄斑区形态可
双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出黄斑形态可
===双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑形态可
右角膜清!左角膜鼻侧白斑!双虹膜可瞳孔圆!右视盘边清色红C/D=0.2A:V=2:3未见

双视盘边清色红!右视盘前可见玻璃体漂浮物C/D=0.2A:V=2:3未见明确出血及渗出黄斑基本正常
===双视盘边清色红!右视盘前可见玻璃体漂浮物C/D=0.2A/V=2:3未见明确出血及渗出黄斑基本正常
双视盘界清色可C/D=0.4动脉细A:V=1:2黄斑大致正常
===双视盘界清色可C/D=0.4动脉细A/V=1:2黄斑大致正常
左底像模糊视盘界清色可C/D=0.4动脉细A:V=1:3黄斑色黑
===左底像模糊视盘界清色可C/D=0.4动脉细A/V=1:3黄斑色黑
右视网膜片状出血视乳头水肿!左C/D=0.3盘沿形态可A:V=1:2交叉压迫征+黄斑区未见明显出血渗出周边视网膜未见明显出血渗出
===右视网膜片状出血视乳头水肿!左C/D=0.3盘沿形态可A/V=1:2交叉压迫征+黄斑区未见明显出血渗出周边视网膜未见明显出血渗出
双视盘水肿A:V=1:3黄斑及视网膜散在出血及渗出
===双视盘水肿A/V=1:3黄斑及视网膜散在出血及渗出
双角膜清瞳孔散大!双视盘边清色红C:/D=0.2A:V=1:2动脉反光增强未见明显微血管瘤出血及渗出黄斑结构可
===双角膜清瞳孔散大!双视盘边清色红C:/D=0.2A/V=1:2动脉反光增强未见明显微血管瘤出血及渗出黄斑结构可
右视盘水肿边界不清可见视盘下方出血A:V=1:3可见动静脉交叉压迫征!左底未见明显异常
===右视盘水肿边界不清可见视盘下方出血A/V=1:3可见动静脉交叉压迫征!左底未见明显异常
双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出!右黄斑前膜
===双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出!右黄斑前膜
双视盘边清色正C:D!右0.5!左0.6视网膜血管走行正常A:V=2:3黄斑中心凹反光可见
===双视盘边清色正C:D!右0.5!左0.6视网膜血管走行正常A/V=2:3黄斑中心凹反光可见
双视盘边清色正!右C/D=0.7!左C/D=0.8视网膜血管走行正常动脉管径偏细A:V≈1:3黄斑区未见明显异常
===双视盘边清色正!右C/D=0.7!左C/D=0.8视网膜血管走行正常动脉管径偏细A/V=≈1:3黄斑区未见明显异常
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=2:3!双黄斑区形态可
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=2:3!双黄斑区

双角膜清!双视盘边清色红!右C/D=0.6!左C/D=0.4A:V=2:3未见明确出血及渗出黄斑形态可
===双角膜清!双视盘边清色红!右C/D=0.6!左C/D=0.4A/V=2:3未见明确出血及渗出黄斑形态可
双视盘边清色红C/D=0.2A:V=1:2未见动静脉压迫征黄斑基本正常
===双视盘边清色红C/D=0.2A/V=1:2未见动静脉压迫征黄斑基本正常
双视盘边清色红C/D=0.2A:V=2:3未见动静脉压迫征黄斑基本正常
===双视盘边清色红C/D=0.2A/V=2:3未见动静脉压迫征黄斑基本正常
双视盘边清色红C/D=0.2A:V=1:2未见明确出血及渗出黄斑基本正常
===双视盘边清色红C/D=0.2A/V=1:2未见明确出血及渗出黄斑基本正常
右角膜清瞳孔不规则眼底像略模糊视盘边清色红C/D=0.5A:V=1:2未见明确出血及渗出
===右角膜清瞳孔不规则眼底像略模糊视盘边清色红C/D=0.5A/V=1:2未见明确出血及渗出
双视盘边清色红C/D=0.2A:V=1:2未见明确出血及渗出黄斑形态可
===双视盘边清色红C/D=0.2A/V=1:2未见明确出血及渗出黄斑形态可
右晶状体混浊!左视盘边清色可C/D=0.2A:V=1:2黄斑形态可细节窥不清
===右晶状体混浊!左视盘边清色可C/D=0.2A/V=1:2黄斑形态可细节窥不清
双视乳头边清色正动脉细A:V=1:2动脉反光强轻度动静脉交叉压迫网膜未见渗出及出血未见微血管瘤
===双视乳头边清色正动脉细A/V=1:2动脉反光强轻度动静脉交叉压迫网膜未见渗出及出血未见微血管瘤
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区形态可
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:2!双黄斑区形态可
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=2:3!右黄斑区可疑膜状增生!左黄斑区形态可
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=2:3!右黄斑区可疑膜状增生!左黄斑区形态可
双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出黄斑形态可
===双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑形态可
双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出黄斑形态可
===双视盘边清色红C/D=0.2A/

双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=2:3!双黄斑区形态可
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=2:3!双黄斑区形态可
双视盘颜色盘沿形态可C/D=0.4!双视网膜血管A:V=1:2!右黄斑区形态可!左黄斑前膜状增殖
===双视盘颜色盘沿形态可C/D=0.4!双视网膜血管A/V=1:2!右黄斑区形态可!左黄斑前膜状增殖
双C/D0.3盘沿形态可A:V=1:2!右黄斑及视网膜未见明显出血及渗出!左颞上方片状出血
===双C/D0.3盘沿形态可A/V=1:2!右黄斑及视网膜未见明显出血及渗出!左颞上方片状出血
双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出黄斑形态可
===双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑形态可
双视盘边清色红C/D=0.2A:V=2:3!右颞上方血管附近可见视网膜前皱褶!左视盘下方可见视网膜线状出血
===双视盘边清色红C/D=0.2A/V=2:3!右颞上方血管附近可见视网膜前皱褶!左视盘下方可见视网膜线状出血
双视盘边清色红!右视盘颞上方可见线状出血A:V=2:3
===双视盘边清色红!右视盘颞上方可见线状出血A/V=2:3
双视盘界清色可C/D=0.4A:V=1:2视网膜出血及渗出
===双视盘界清色可C/D=0.4A/V=1:2视网膜出血及渗出
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区形态可
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:2!双黄斑区形态可
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:3!双视网膜散在出血软性渗出
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:3!双视网膜散在出血软性渗出
左底局限不清晶体后囊可疑混浊!双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区形态可
===左底局限不清晶体后囊可疑混浊!双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:2!双黄斑区形态可
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=2:3!双黄斑区及周边视网膜未见明显出血渗出
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=2:3!双黄斑区及周边视网膜未见明显出血渗出
右颞上方结膜炎出血!双视盘颜色盘沿形态

双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出黄斑基本正常
===双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑基本正常
双底像模糊!双视盘边清色红C/D=0.2A:V=1:2未见动静脉压迫征黄斑基本正常
===双底像模糊!双视盘边清色红C/D=0.2A/V=1:2未见动静脉压迫征黄斑基本正常
双底轻度模糊!双视盘下方颜色淡盘沿偏窄!双视网膜血管A:V=1:2!双黄斑区脉络膜萎缩
===双底轻度模糊!双视盘下方颜色淡盘沿偏窄!双视网膜血管A/V=1:2!双黄斑区脉络膜萎缩
双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A:V=1:2!双黄斑区形态可未见明显出血渗出
===双视盘颜色盘沿形态可C/D=0.3!双视网膜血管A/V=1:2!双黄斑区形态可未见明显出血渗出
左C/D=0.3盘沿形态可A:V=1:2黄斑区未见明显出血渗出周边视网膜未见明显出血渗出!右底看不清!双外眼像晶体混浊!右为重
===左C/D=0.3盘沿形态可A/V=1:2黄斑区未见明显出血渗出周边视网膜未见明显出血渗出!右底看不清!双外眼像晶体混浊!右为重
双C/D0.3盘沿形态可A:V=1:2黄斑及视网膜未见明显出血及渗出
===双C/D0.3盘沿形态可A/V=1:2黄斑及视网膜未见明显出血及渗出
右C/D0.3盘沿形态可A:V1:2黄斑点状渗出
===右C/D0.3盘沿形态可A/V=1:2黄斑点状渗出
双角膜清!双视盘边清色红C/D=0.5A:V=2:3未见明确出血及渗出黄斑形态可
===双角膜清!双视盘边清色红C/D=0.5A/V=2:3未见明确出血及渗出黄斑形态可
双视盘边清色红C/D=0.2A:V=2:3未见动静脉压迫征黄斑基本正常
===双视盘边清色红C/D=0.2A/V=2:3未见动静脉压迫征黄斑基本正常
双视盘边清色红C/D=0.2A:V=2:3未见明确出血及渗出!右黄斑区周围可见皱褶!左黄斑基本正常
===双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出!右黄斑区周围可见皱褶!左黄斑基本正常
双视盘边清色红C/D=0.3A:V=1:2未见明确出血及渗出黄斑出色暗未见明确病灶及形态改变
===双视盘边清色红C/D=0.3A/V=1:2未见明确出血及渗出黄斑出色暗未见明确病灶及形态改变
双豹纹状眼底盘周见萎缩弧视网膜血管走行正常A:V约1:3

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('\d:\d')]['影描_gai']

## 添加新标签

In [35]:
df_ym.insert(df_ym.columns.get_loc('影描_gai')+1,'right_动静脉管径比',None)
df_ym.insert(df_ym.columns.get_loc('影描_gai')+1,'left_动静脉管径比',None)

In [ ]:
a = '双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V=1:2!右黄斑区可见处未.'

In [36]:
# 添加脉络膜Coats病的解析标签
def add_new_label_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if len(re.compile('A/V').findall(str_i)) > 0:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            for k in range(len(str_i_list)):
                if (len(re.compile('A/V=\d+:\d+').findall(str_i_list[k]))>0) and (str.find(str_i_list[k],'双')!=-1):
#                     print(str_i_list[k])
                    index_start = [i.start() for i in re.finditer('A/V=\d+:\d+',str_i_list[k])] # 起始索引
                    index_end = [i.end() for i in re.finditer('A/V=\d+:\d+',str_i_list[k])] # 终止索引 
#                     print(index_start,index_end)
                    df.at[i,'left_动静脉管径比'] = str_i_list[k][index_start[0]:index_end[0]] # 截取杯盘比
                    df.at[i,'right_动静脉管径比'] = str_i_list[k][index_start[0]:index_end[0]] 
                elif (len(re.compile('A/V=\d+:\d+').findall(str_i_list[k]))>0) and (str.find(str_i_list[k],'左')!=-1):
                    index_start = [i.start() for i in re.finditer('A/V=\d+:\d+',str_i_list[k])] # 起始索引
                    index_end = [i.end() for i in re.finditer('A/V=\d+:\d+',str_i_list[k])] # 终止索引 
                    df.at[i,'left_动静脉管径比'] = str_i_list[k][index_start[0]:index_end[0]] # 截取杯盘比
                elif (len(re.compile('A/V=\d+:\d+').findall(str_i_list[k]))>0) and (str.find(str_i_list[k],'右')!=-1):
                    index_start = [i.start() for i in re.finditer('A/V=\d+:\d+',str_i_list[k])] # 起始索引
                    index_end = [i.end() for i in re.finditer('A/V=\d+:\d+',str_i_list[k])] # 终止索引                  
                    df.at[i,'right_动静脉管径比'] = str_i_list[k][index_start[0]:index_end[0]] 
                else:
                    pass
        else:
            pass

In [37]:
add_new_label_fuc(df_ym)   # 调用添加新标签的函数

双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常
[28] [35]
双底像模糊隐见视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[34] [41]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区散在玻璃膜疣余大致正常细节窥不清
[33] [40]
双视网膜血管A/V=1:2
[6] [13]
双视网膜血管A/V=2:3
[6] [13]
双A/V=1:2黄斑区未见明显出血渗出周边视网膜未见明显出血渗出
[1] [8]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出
[14] [21]
双视盘边清色红C/D=0.2A/V=1:2未见明确出血及渗出黄斑基本正常
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V=1:2
[30] [37]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清盘沿形态未见明确异常视网膜动脉管径普

双视盘水肿A/V=1:3黄斑及视网膜散在出血及渗出
[5] [12]
双视盘边清色红C:/D=0.2A/V=1:2动脉反光增强未见明显微血管瘤出血及渗出黄斑结构可
[15] [22]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3黄斑区周边可见散在玻璃膜疣余可见处未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常
[12] [19]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光增强视网膜静脉迂曲
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区模糊可见处未见明显异常
[28] [35]
双视盘边清色淡盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象
[28] [35]
双视网膜血管A/V=2:3
[6] [13]
双视网膜血管A/V=1:2
[6] [13]
双视网膜血管A/V=1:2
[6] [13]
双视网膜血管A/V=1:2
[6] [13]
双底视盘边界清色淡红C/D=0.2A/V=1:2部分区域可见动静脉交叉压迫黄斑大致正常
[17] [24]
双视盘边清色红C/D=0.2A/V=2:3黄斑结构可
[14] [21]
双视盘边清色红C/D=0.2A/V=1:2可见动静脉交叉压迫征未见明确出血及渗出黄斑形态可
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3可见微血管瘤视网膜出血及少许硬渗黄斑基本正常
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强

双视网膜血管A/V=1:2
[6] [13]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑区深层新生血管
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑形态可
[14] [21]
双视盘边清色红C/D=0.2A/V=1:2未见明确出血及渗出黄斑色暗细节窥不清
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑基本正常
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑基本正常
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径略细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[26] [33]
双视网膜动脉管径普遍变细A/V=1:2
[12] [19]
双视盘界清色可动脉细反光强A/V=1:2动静脉交叉压迫征
[13] [20]
双视网膜血管A/V=1:2
[6] [13]
双视网膜血管A/V=1:2
[6] [13]
双视网膜血管A/V=1:2
[6] [13]
双视盘边清色红C/D=0.2A/V=1:2未见明显出血及渗出黄斑结构可
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑区可见少许玻璃膜疣
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见动静脉压迫征黄斑基本正常
[14] [21]
双视盘边清色红C/D=0.1A/V=2:3未见明确出血及渗出黄斑基本正常
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见动静脉压迫征黄斑基本正常
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2

双视盘边界可色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑区形态可
[15] [22]
双A/V=1:2黄斑区未见明显出血渗出
[1] [8]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常颞上视网膜见黄白色点状陈旧病灶未见未见微血管瘤
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区结构未见明显异常
[28] [35]
双轻度豹纹状眼底盘沿形态未见明确异常杯盘比0.6盘周未见萎缩弧视网膜血管走行直A/V=1:2未见异常新生血管
[39] [46]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2动静脉出现交叉压迹现象黄斑区可见处未见明显异常
[28] [35]
双视网膜动脉管径普遍变细A/V=1:2动脉管壁反光增强
[12] [19]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常后极部视网膜散在黄白点状病灶
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强
[28] [35]
双视盘颜色盘沿形态可A/V=1:2
[10] [17]
双视网膜血管A/V=2:3
[6] [13]
双A/V=1:2黄斑区未见明显出血渗出周边视网膜未见明显出血渗出
[1] [8]
双视盘边清色红C/D=0.2A/V=1:2
[14] [21]
双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细局部A/V=1:2黄斑区结构未见明显异

双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑基本正常
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑基本正常
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2动静脉出现交叉压迹现象
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区看不清
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区结构未见明显异常
[12] [19]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2动静脉出现交叉压迹现象黄斑区可见处未见明显异常
[28] [35]
双底动脉反光增强A/V=1:3-4可见交叉压迹
[8] [15]
双A/V=1:2动静脉压迫征+
[1] [8]
双C/D0.3盘沿形态可A/V=1:2黄斑及视网膜未见明显出血及渗出
[12] [19]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑基本正常
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径变细A/V=1:2视网膜动脉管壁反光增强
[26] [33]
双视盘边清色正盘沿形态未见明确异常视乳头色淡红视网膜动脉管径普遍变细A/V=2:3视网膜动脉管壁反光增强动静脉未出现交叉压迹现象黄斑区结构未见明显异常
[34] [41]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A

双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑形态可
[14] [21]
双视盘边清色红C/D=0.3A/V=2:3未见动静脉压迫征黄斑色暗
[14] [21]
双视盘边清色红C/D=0.2A/V=1:2未见动静脉压迫征黄斑基本正常
[14] [21]
双豹纹状眼底视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区欠清晰
[33] [40]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常颞上方裂隙状RNFLD视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区模糊可见处未见明显异常
[39] [46]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2黄斑区暗窥不清
[28] [35]
双视网膜血管A/V=1:2
[6] [13]
双视网膜血管A/V=1:2
[6] [13]
双C/D0.3盘沿形态可A/V=1:2黄斑及视网膜未见明显出血及渗出
[12] [19]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出
[14] [21]
双视盘边清色红C/D=0.2A/V=2:3未见明确出血及渗出黄斑区可见黄白色点状病变
[14] [21]
双视盘边清色可C/D=0.2A/V=2:3未见明确出血及渗出
[14] [21]
双视盘边清色正盘沿形态未见明确异常视网膜血管走行正常A/V=1:2未见异常新生血管未见神经纤维层缺损黄斑区未见明显异常
[26] [33]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强动静脉出现交叉压迹现象黄斑区结构未见明显异常
[28] [35]
双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:2视网膜动脉管壁反光增强黄斑区

### '影描_gai'字段——含“Coats病”标签的数据容量

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('A/V=\d+:\d+')]['影描_gai']

In [38]:
print("'影描_gai'字段——含A/V——数据容量:{}".format(df_ym[df_ym['影描_gai'].str.contains('A/V')].shape[0] ))  # 含青光眼的数据容量

'影描_gai'字段——含A/V——数据容量:1014


In [39]:
df_ym['left_动静脉管径比'].value_counts()  # C/D=0.5--

A/V=1:2     709
A/V=2:3     185
A/V=1:3      45
A/V=1:4       2
A/V=1:32      1
Name: left_动静脉管径比, dtype: int64

In [40]:
df_ym['right_动静脉管径比'].value_counts()

A/V=1:2     694
A/V=2:3     191
A/V=1:3      44
A/V=1:32      2
A/V=1:4       2
Name: right_动静脉管径比, dtype: int64

In [ ]:
df_ym[df_ym['left_动静脉管径比'].isin(['A/V=1:32'])][['影像描述','影描_gai']]

In [41]:
df_ym_jiao = df_ym[['影像描述','影描_gai','left_动静脉管径比','right_动静脉管径比']].drop_duplicates(keep='first')  # 去重
## 患者维度
print("'影描_gai'字段去重后——数据容量:{}".format(df_ym_jiao.shape[0]) )  # '影描_gai'字段去重后的数据容量
df_ym_jiao[:5]

'影描_gai'字段去重后——数据容量:122181


,影像描述,影描_gai,left_动静脉管径比,right_动静脉管径比
0,"视盘边清色淡,右眼C/D=0.7,颞侧盘沿消失,双眼黄斑中心光未见,视网膜动脉细,反光强,交...",双视盘边清色淡!右C/D=0.7颞侧盘沿消失!双黄斑中心光未见视网膜动脉细反光强交叉征+余视...,None,None
1,"双眼视盘边清色正,盘沿形态未见明确异常;视网膜血管走行正常,未见异常新生血管;右眼视网膜散在...",双视盘边清色正盘沿形态未见明确异常视网膜血管走行正常未见异常新生血管!右视网膜散在玻璃膜疣黄...,None,None
2,"双眼视盘边清色正,盘沿形态未见明确异常;视网膜动脉管径普遍变细,A/V=1:3,视网膜动脉管...",双视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光增强...,A/V=1:3,A/V=1:3
3,"双眼后极部散在点片状出血灶,黄白色病灶",双后极部散在点片状出血灶黄白色病灶,None,None
4,"可见双眼视盘边清色淡,黄斑中心光未见,视网膜呈豹纹状。",双可见!双视盘边清色淡黄斑中心光未见视网膜呈豹纹状,None,None


In [42]:
## 患者维度——含白内障
### '影描_gai'字段——含“白内障”标签的数据容量——去重后，唯一标签
df_ym_jiao_notduplicate = df_ym_jiao[df_ym_jiao['影描_gai'].str.contains('A/V=')]   # '影描_gai'字段去重后——含白内障的数据容量
print("'影描_gai'字段去重后_含C/D——数据容量:{}".format(df_ym_jiao_notduplicate.shape[0]))

'影描_gai'字段去重后_含C/D——数据容量:679


In [43]:
df_last_01 = df_ym_jiao.sample(n=5000,replace=False,random_state=None,axis=0)

In [44]:
df_last_01[df_last_01['影描_gai'].str.contains('A/V')].shape[0]

30

In [45]:
output_path = r'/Users/edz/Documents/yandi_data/导出数据/影像描述'
output_file = r'/动静脉管径比' + str(df_ym[df_ym['影描_gai'].str.contains('A/V=')].shape[0]) +'_20200401.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df_last_01.to_excel(writer,sheet_name='含动静脉管径比_5000',index=None)
df_ym_jiao_notduplicate.to_excel(writer,sheet_name='含动静脉管径比_去重_'+ str(df_ym_jiao_notduplicate.shape[0]),index=None)
df_ym_jiao.to_excel(writer,sheet_name='影描_gai_去重_'+str(df_ym_jiao.shape[0]),index=None)
writer.save()

In [46]:
# 输出全量250625的糖网标签
df_ym_quan = df_ym[['来自检查id','影描_gai','left_动静脉管径比','right_动静脉管径比']]

In [47]:
df_ym_quan.shape[0]

250625

In [48]:
# 导出新增疾病标签
output_path = r'/Users/edz/Documents/yandi_data/AI_data/zn_ai_影像描述标签'
output_file = r'/动静脉管径比_quan_250625.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df_ym_quan.to_excel(writer,sheet_name='动静脉管径比'+str(df_ym_quan.shape[0]),index=None)
writer.save()